In [124]:
import os
import warnings
import pandas as pd
import datetime
import holidays

In [125]:
# 분석에 문제 없는 경고 메세지 숨기기
warnings.filterwarnings("ignore")

# 접근 기본 디렉토리 변경 (사람마다 다름)
os.chdir('D:/Jupyter/cap1')

# 정제 완료 데이터 가져오기

In [126]:
air_pollution = pd.read_csv('./final_air_pollution(2017~2023).csv', encoding = 'utf-8')
weather = pd.read_csv('./final_weather(2017~2023).csv', encoding = 'utf-8')
local_people = pd.read_csv('./final_local_people(2017~2022).csv', encoding = 'utf-8')

In [127]:
air_pollution['기준일'] = pd.to_datetime(air_pollution['기준일'])
air_pollution.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50588 entries, 0 to 50587
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   기준일           50588 non-null  datetime64[ns]
 1   시간            50588 non-null  int64         
 2   평균 미세먼지(㎍/㎥)  50588 non-null  int64         
dtypes: datetime64[ns](1), int64(2)
memory usage: 1.2 MB


In [128]:
weather['기준일'] = pd.to_datetime(weather['기준일'])
weather.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54535 entries, 0 to 54534
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   기준일      54535 non-null  datetime64[ns]
 1   시간       54535 non-null  int64         
 2   기온(°C)   54369 non-null  float64       
 3   풍속(m/s)  54207 non-null  float64       
 4   강수량(mm)  53941 non-null  float64       
 5   습도(%)    54400 non-null  float64       
dtypes: datetime64[ns](1), float64(4), int64(1)
memory usage: 2.5 MB


In [129]:
local_people['기준일'] = pd.to_datetime(local_people['기준일'])
local_people.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1306800 entries, 0 to 1306799
Data columns (total 5 columns):
 #   Column  Non-Null Count    Dtype         
---  ------  --------------    -----         
 0   자치구코드   1306800 non-null  int64         
 1   행정구     1306800 non-null  object        
 2   기준일     1306800 non-null  datetime64[ns]
 3   시간      1306800 non-null  int64         
 4   총생활인구수  1306800 non-null  int64         
dtypes: datetime64[ns](1), int64(3), object(1)
memory usage: 49.9+ MB


# 휴일, 요일 나타내기

In [130]:
gu = pd.DataFrame(columns=['temp', '자치구코드'])
gu_code = local_people['자치구코드'].unique()
gu['자치구코드'] = sorted(gu_code)

In [131]:
# 한국 2017~2022년 휴일 객체 생성 
kr_holidays = holidays.KR(years = range(2017, 2023))

# 2017년부터 2022년 데이터 생성
date_list = pd.date_range('20170101', '20221231')

# 휴일을 정리할 dataframe 생성
holiday_df = pd.DataFrame(columns=['temp', '기준일', 'holiday'])
holiday_df['기준일'] = sorted(date_list)

# 월화수목금토일 분류
holiday_df['day'] = holiday_df['기준일'].dt.weekday

# 공휴일 반영
holiday_df['holiday'] = holiday_df.기준일.apply(lambda x: 2 if x in kr_holidays else 0)

# 공휴일 지정된 데이터만 정리하기
real_holiday = holiday_df[holiday_df['holiday'] == 2]

In [132]:
# 공휴일 전날 데이터만 정리하기
before_holiday = list(real_holiday.index-1)
before_holiday = [a for a in before_holiday if a >0]

# 공휴일 전날 데이터의 holiday 값을 1로 변경
for i in before_holiday:
    holiday_df.at[i, 'holiday'] = 1

# 금요일 데이터
fri = holiday_df[holiday_df['day'] == 4]
fri['holiday'] = 1

# holiday 1인 데이터를 합치기
for i in list(fri.index):
    holiday_df.at[i, 'holiday'] = fri.at[i, 'holiday']

In [133]:
# 공휴일 holiday값들 2로 바꾸기
sat = holiday_df[holiday_df['day'] == 5]
sat['holiday'] = 2

sun = holiday_df[holiday_df['day'] == 6]
sun['holiday'] = 2

In [134]:
# holiday 2인 데이터를 합치기
for i in list(sat.index):
    holiday_df.at[i, 'holiday'] = sat.at[i, 'holiday']
    
for i in list(sun.index):
    holiday_df.at[i, 'holiday'] = sun.at[i, 'holiday']
    
for i in list(real_holiday.index):
    holiday_df.at[i, 'holiday'] = real_holiday.at[i, 'holiday']

holiday_df

,temp,기준일,holiday,day
0,NaN,2017-01-01,2,6
1,NaN,2017-01-02,0,0
2,NaN,2017-01-03,0,1
3,NaN,2017-01-04,0,2
4,NaN,2017-01-05,0,3
...,...,...,...,...
2186,NaN,2022-12-27,0,1
2187,NaN,2022-12-28,0,2
2188,NaN,2022-12-29,0,3
2189,NaN,2022-12-30,1,4


In [135]:
hour_list = []

for i in range(24):
    hour_list.append(i)
    
total_hour_list = pd.DataFrame(columns=['temp', '시간'])
total_hour_list['시간'] = sorted(hour_list)
total_hour_list

,temp,시간
0,NaN,0
1,NaN,1
2,NaN,2
3,NaN,3
4,NaN,4
5,NaN,5
6,NaN,6
7,NaN,7
8,NaN,8
9,NaN,9


In [136]:
holiday_df = pd.merge(holiday_df, total_hour_list, how = 'outer')
holiday_df

,temp,기준일,holiday,day,시간
0,NaN,2017-01-01,2,6,0
1,NaN,2017-01-01,2,6,1
2,NaN,2017-01-01,2,6,2
3,NaN,2017-01-01,2,6,3
4,NaN,2017-01-01,2,6,4
...,...,...,...,...,...
52579,NaN,2022-12-31,2,5,19
52580,NaN,2022-12-31,2,5,20
52581,NaN,2022-12-31,2,5,21
52582,NaN,2022-12-31,2,5,22


In [137]:
holiday_gu = pd.merge(holiday_df, gu, how = 'outer')
holiday_gu = holiday_gu.drop(['temp'], axis = 1)
holiday_gu

,기준일,holiday,day,시간,자치구코드
0,2017-01-01,2,6,0,11110
1,2017-01-01,2,6,0,11140
2,2017-01-01,2,6,0,11170
3,2017-01-01,2,6,0,11200
4,2017-01-01,2,6,0,11215
...,...,...,...,...,...
1314595,2022-12-31,2,5,23,11620
1314596,2022-12-31,2,5,23,11650
1314597,2022-12-31,2,5,23,11680
1314598,2022-12-31,2,5,23,11710


# 데이터 합치기

In [138]:
# merge 선행
total = pd.merge(holiday_gu, weather, how = 'left')
total = pd.merge(total, air_pollution, how = 'left')
total = pd.merge(total, local_people, how = 'left')

# 결측값 제거
total = total.fillna(method='ffill')

In [139]:
total.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1314600 entries, 0 to 1314599
Data columns (total 12 columns):
 #   Column        Non-Null Count    Dtype         
---  ------        --------------    -----         
 0   기준일           1314600 non-null  datetime64[ns]
 1   holiday       1314600 non-null  int64         
 2   day           1314600 non-null  int64         
 3   시간            1314600 non-null  int64         
 4   자치구코드         1314600 non-null  int64         
 5   기온(°C)        1314600 non-null  float64       
 6   풍속(m/s)       1314600 non-null  float64       
 7   강수량(mm)       1314600 non-null  float64       
 8   습도(%)         1314600 non-null  float64       
 9   평균 미세먼지(㎍/㎥)  1314600 non-null  float64       
 10  행정구           1314600 non-null  object        
 11  총생활인구수        1314600 non-null  float64       
dtypes: datetime64[ns](1), float64(6), int64(4), object(1)
memory usage: 130.4+ MB


In [140]:
total['year'] = total['기준일'].dt.year
total['month'] = total['기준일'].dt.month

total['총생활인구수'] = total['총생활인구수'].astype('int64')

In [141]:
total_columns = ['기준일', 'holiday', 'year', 'month', 'day', '시간', '자치구코드', '행정구', '기온(°C)', '풍속(m/s)', '강수량(mm)', '습도(%)', '평균 미세먼지(㎍/㎥)', '총생활인구수']
total = total.reindex(columns = total_columns)
total.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1314600 entries, 0 to 1314599
Data columns (total 14 columns):
 #   Column        Non-Null Count    Dtype         
---  ------        --------------    -----         
 0   기준일           1314600 non-null  datetime64[ns]
 1   holiday       1314600 non-null  int64         
 2   year          1314600 non-null  int64         
 3   month         1314600 non-null  int64         
 4   day           1314600 non-null  int64         
 5   시간            1314600 non-null  int64         
 6   자치구코드         1314600 non-null  int64         
 7   행정구           1314600 non-null  object        
 8   기온(°C)        1314600 non-null  float64       
 9   풍속(m/s)       1314600 non-null  float64       
 10  강수량(mm)       1314600 non-null  float64       
 11  습도(%)         1314600 non-null  float64       
 12  평균 미세먼지(㎍/㎥)  1314600 non-null  float64       
 13  총생활인구수        1314600 non-null  int64         
dtypes: datetime64[ns](1), float64(5), int64(7), object

In [142]:
total.columns = ['date', 'holiday', 'year', 'month', 'day', 'hour', 'gu_code', 'gu', 'temp', 'wind', 'rain', 'humidity', 'dust', 'people']
total

,date,holiday,year,month,day,hour,gu_code,gu,temp,wind,rain,humidity,dust,people
0,2017-01-01,2,2017,1,6,0,11110,종로구,2.0,0.4,0.0,80.4,54.0,283391
1,2017-01-01,2,2017,1,6,0,11140,중구,2.0,0.4,0.0,80.4,54.0,189272
2,2017-01-01,2,2017,1,6,0,11170,용산구,2.0,0.4,0.0,80.4,54.0,288680
3,2017-01-01,2,2017,1,6,0,11200,성동구,2.0,0.4,0.0,80.4,54.0,313852
4,2017-01-01,2,2017,1,6,0,11215,광진구,2.0,0.4,0.0,80.4,54.0,382976
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1314595,2022-12-31,2,2022,12,5,23,11620,관악구,-1.0,1.0,0.0,60.9,70.0,468533
1314596,2022-12-31,2,2022,12,5,23,11650,서초구,-1.0,1.0,0.0,60.9,70.0,464794
1314597,2022-12-31,2,2022,12,5,23,11680,강남구,-1.0,1.0,0.0,60.9,70.0,626217
1314598,2022-12-31,2,2022,12,5,23,11710,송파구,-1.0,1.0,0.0,60.9,70.0,726058


In [58]:
# total.to_csv('./final_merge_data.csv', index=False, encoding='utf-8-sig')

# 생활인구에서 상주인구 빼기

In [143]:
total

,date,holiday,year,month,day,hour,gu_code,gu,temp,wind,rain,humidity,dust,people
0,2017-01-01,2,2017,1,6,0,11110,종로구,2.0,0.4,0.0,80.4,54.0,283391
1,2017-01-01,2,2017,1,6,0,11140,중구,2.0,0.4,0.0,80.4,54.0,189272
2,2017-01-01,2,2017,1,6,0,11170,용산구,2.0,0.4,0.0,80.4,54.0,288680
3,2017-01-01,2,2017,1,6,0,11200,성동구,2.0,0.4,0.0,80.4,54.0,313852
4,2017-01-01,2,2017,1,6,0,11215,광진구,2.0,0.4,0.0,80.4,54.0,382976
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1314595,2022-12-31,2,2022,12,5,23,11620,관악구,-1.0,1.0,0.0,60.9,70.0,468533
1314596,2022-12-31,2,2022,12,5,23,11650,서초구,-1.0,1.0,0.0,60.9,70.0,464794
1314597,2022-12-31,2,2022,12,5,23,11680,강남구,-1.0,1.0,0.0,60.9,70.0,626217
1314598,2022-12-31,2,2022,12,5,23,11710,송파구,-1.0,1.0,0.0,60.9,70.0,726058


In [144]:
total.groupby(['hour']).mean()['people'].sort_values()

hour
3     415694.745212
4     415709.208307
2     415830.688599
5     415928.105906
1     416070.063003
0     416488.139461
6     416943.452597
23    417412.028462
7     420876.295080
22    420940.881972
21    423874.062236
20    426267.870580
8     426279.773181
19    428755.568763
9     430084.033209
10    432079.183770
18    432500.841168
11    433197.764765
12    434210.545431
17    434906.950324
13    435214.074724
16    435536.822802
14    435773.217216
15    435827.198813
Name: people, dtype: float64

새벽 3시가 사람이 가장 적다는 것을 알 수 있다. -> 새벽 3시 기준으로 유동인구 파악

In [145]:
resident = total[['gu_code', 'date', 'hour', 'people']]
resident = resident[resident['hour']==3]
resident = resident.reset_index()
resident = resident.drop(['hour', 'index'], axis = 1)
resident

,gu_code,date,people
0,11110,2017-01-01,227495
1,11140,2017-01-01,158427
2,11170,2017-01-01,270181
3,11200,2017-01-01,312212
4,11215,2017-01-01,381230
...,...,...,...
54770,11620,2022-12-31,487414
54771,11650,2022-12-31,451885
54772,11680,2022-12-31,637171
54773,11710,2022-12-31,720087


In [157]:
a = total[total['date']=='2022-12-31']
a[a['gu_code']==11740]

,date,holiday,year,month,day,hour,gu_code,gu,temp,wind,rain,humidity,dust,people
1264206,2022-12-31,2,2022,12,5,0,11740,강동구,-5.5,0.6,0.0,49.3,41.0,523283
1266397,2022-12-31,2,2022,12,5,1,11740,강동구,-5.9,0.3,0.0,48.5,41.0,524804
1268588,2022-12-31,2,2022,12,5,2,11740,강동구,-6.0,0.7,0.0,58.4,48.0,525571
1270779,2022-12-31,2,2022,12,5,3,11740,강동구,-5.8,0.3,0.0,53.6,38.0,524856
1272970,2022-12-31,2,2022,12,5,4,11740,강동구,-5.5,0.5,0.0,47.9,35.0,524186
1275161,2022-12-31,2,2022,12,5,5,11740,강동구,-4.9,1.0,0.0,45.0,39.0,524117
1277352,2022-12-31,2,2022,12,5,6,11740,강동구,-4.8,0.8,0.0,44.8,41.0,520566
1279543,2022-12-31,2,2022,12,5,7,11740,강동구,-4.6,0.6,0.0,42.6,40.0,515920
1281734,2022-12-31,2,2022,12,5,8,11740,강동구,-4.6,0.5,0.0,47.7,29.0,511133
1283925,2022-12-31,2,2022,12,5,9,11740,강동구,-4.1,0.1,0.0,49.0,34.0,503848


In [146]:
hour = holiday_gu[['기준일', '시간', '자치구코드']]
hour.columns = ['date', 'hour', 'gu_code']
hour

,date,hour,gu_code
0,2017-01-01,0,11110
1,2017-01-01,0,11140
2,2017-01-01,0,11170
3,2017-01-01,0,11200
4,2017-01-01,0,11215
...,...,...,...
1314595,2022-12-31,23,11620
1314596,2022-12-31,23,11650
1314597,2022-12-31,23,11680
1314598,2022-12-31,23,11710


In [147]:
resident = pd.merge(resident, hour)
resident = resident.sort_values(['gu_code', 'hour', 'date'])

,gu_code,date,people,hour
0,11110,2017-01-01,227495,0
600,11110,2017-01-02,209421,0
1200,11110,2017-01-03,222868,0
1800,11110,2017-01-04,225665,0
2400,11110,2017-01-05,228413,0
...,...,...,...,...
1312199,11740,2022-12-27,536142,23
1312799,11740,2022-12-28,536029,23
1313399,11740,2022-12-29,536767,23
1313999,11740,2022-12-30,532745,23


In [151]:
total = total.sort_values(['gu_code', 'hour', 'date'])
total = total.reset_index().drop(['index'], axis = 1)
total

,date,holiday,year,month,day,hour,gu_code,gu,temp,wind,rain,humidity,dust,people
0,2017-01-01,2,2017,1,6,0,11110,종로구,2.0,0.4,0.0,80.4,54.0,283391
1,2017-01-02,0,2017,1,0,0,11110,종로구,5.8,0.5,0.0,71.9,62.0,211489
2,2017-01-03,0,2017,1,1,0,11110,종로구,3.5,1.0,0.0,74.0,126.0,228204
3,2017-01-04,0,2017,1,2,0,11110,종로구,3.5,0.9,0.0,69.9,35.0,231314
4,2017-01-05,0,2017,1,3,0,11110,종로구,3.6,0.7,0.0,47.4,40.0,234464
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1314595,2022-12-27,0,2022,12,1,23,11740,강동구,-3.1,0.6,0.0,57.4,41.0,531021
1314596,2022-12-28,0,2022,12,2,23,11740,강동구,-6.2,1.4,0.0,44.6,32.0,531043
1314597,2022-12-29,0,2022,12,3,23,11740,강동구,-2.3,0.4,0.0,58.8,33.0,526416
1314598,2022-12-30,1,2022,12,4,23,11740,강동구,-4.9,0.7,0.0,49.6,34.0,519829


In [152]:
resident = resident.sort_values(['gu_code', 'hour', 'date'])
resident = resident.reset_index().drop(['index'], axis = 1)
resident

,gu_code,date,people,hour
0,11110,2017-01-01,227495,0
1,11110,2017-01-02,209421,0
2,11110,2017-01-03,222868,0
3,11110,2017-01-04,225665,0
4,11110,2017-01-05,228413,0
...,...,...,...,...
1314595,11740,2022-12-27,536142,23
1314596,11740,2022-12-28,536029,23
1314597,11740,2022-12-29,536767,23
1314598,11740,2022-12-30,532745,23


In [149]:
floating_population = total - resident
floating_population

,date,day,dust,gu,gu_code,holiday,hour,humidity,month,people,rain,temp,wind,year
0,0 days,NaN,NaN,NaN,0,NaN,0,NaN,NaN,55896,NaN,NaN,NaN,NaN
1,0 days,NaN,NaN,NaN,0,NaN,0,NaN,NaN,2068,NaN,NaN,NaN,NaN
2,0 days,NaN,NaN,NaN,0,NaN,0,NaN,NaN,5336,NaN,NaN,NaN,NaN
3,0 days,NaN,NaN,NaN,0,NaN,0,NaN,NaN,5649,NaN,NaN,NaN,NaN
4,0 days,NaN,NaN,NaN,0,NaN,0,NaN,NaN,6051,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1314595,0 days,NaN,NaN,NaN,0,NaN,0,NaN,NaN,-5121,NaN,NaN,NaN,NaN
1314596,0 days,NaN,NaN,NaN,0,NaN,0,NaN,NaN,-4986,NaN,NaN,NaN,NaN
1314597,0 days,NaN,NaN,NaN,0,NaN,0,NaN,NaN,-10351,NaN,NaN,NaN,NaN
1314598,0 days,NaN,NaN,NaN,0,NaN,0,NaN,NaN,-12916,NaN,NaN,NaN,NaN


In [159]:
floating_population[floating_population['people']<0]

,date,day,dust,gu,gu_code,holiday,hour,humidity,month,people,rain,temp,wind,year
143,0 days,NaN,NaN,NaN,0,NaN,0,NaN,NaN,-7434,NaN,NaN,NaN,NaN
158,0 days,NaN,NaN,NaN,0,NaN,0,NaN,NaN,-8214,NaN,NaN,NaN,NaN
204,0 days,NaN,NaN,NaN,0,NaN,0,NaN,NaN,-2850,NaN,NaN,NaN,NaN
248,0 days,NaN,NaN,NaN,0,NaN,0,NaN,NaN,-847,NaN,NaN,NaN,NaN
288,0 days,NaN,NaN,NaN,0,NaN,0,NaN,NaN,-5989,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1314595,0 days,NaN,NaN,NaN,0,NaN,0,NaN,NaN,-5121,NaN,NaN,NaN,NaN
1314596,0 days,NaN,NaN,NaN,0,NaN,0,NaN,NaN,-4986,NaN,NaN,NaN,NaN
1314597,0 days,NaN,NaN,NaN,0,NaN,0,NaN,NaN,-10351,NaN,NaN,NaN,NaN
1314598,0 days,NaN,NaN,NaN,0,NaN,0,NaN,NaN,-12916,NaN,NaN,NaN,NaN


# 사람 수가 아닌 그 시간대 기준 퍼센트로 비율 나타내기

In [7]:
local_people

,자치구코드,행정구,기준일,시간,총생활인구수
0,11110,종로구,2017-01-01,0,283391
1,11140,중구,2017-01-01,0,189272
2,11170,용산구,2017-01-01,0,288680
3,11200,성동구,2017-01-01,0,313852
4,11215,광진구,2017-01-01,0,382976
...,...,...,...,...,...
1306795,11620,관악구,2022-12-31,23,468533
1306796,11650,서초구,2022-12-31,23,464794
1306797,11680,강남구,2022-12-31,23,626217
1306798,11710,송파구,2022-12-31,23,726058


In [8]:
local_people_temp = local_people[['자치구코드', '기준일', '시간', '총생활인구수']]
local_people_temp

,자치구코드,기준일,시간,총생활인구수
0,11110,2017-01-01,0,283391
1,11140,2017-01-01,0,189272
2,11170,2017-01-01,0,288680
3,11200,2017-01-01,0,313852
4,11215,2017-01-01,0,382976
...,...,...,...,...
1306795,11620,2022-12-31,23,468533
1306796,11650,2022-12-31,23,464794
1306797,11680,2022-12-31,23,626217
1306798,11710,2022-12-31,23,726058


In [9]:
local_people_temp = local_people_temp.pivot(index=['기준일', '시간'], columns = '자치구코드', values = '총생활인구수')
local_people_temp

자치구코드           11110   11140   11170   11200   11215   11230   11260   11290  \
기준일        시간                                                                   
2017-01-01 0   283391  189272  288680  313852  382976  355707  390842  437300   
           1   240461  165315  279707  311892  384657  354840  394269  438263   
           2   231657  160031  274205  311383  382445  354783  395879  439963   
           3   227495  158427  270181  312212  381230  355125  396765  441106   
           4   226370  157119  266860  312450  381829  355685  396992  442033   
...               ...     ...     ...     ...     ...     ...     ...     ...   
2022-12-31 19  265097  229317  281156  314924  356525  320526  351974  402077   
           20  252970  211170  275322  316909  359559  320940  355901  406948   
           21  239792  196161  269746  315531  363770  323086  361863  412726   
           22  233440  181366  262516  313423  365134  324218  365115  418728   
           23  241489  161747  254426  303053  362365  327369  368713  417970   

자치구코드           11305   11320  ...   11500   11530   11545   11560   11590  \
기준일        시간                  ...                                           
2017-01-01 0   340028  320333  ...  562423  405217  196453  422638  417771   
           1   341804  323049  ...  566660  406676  197981  406659  421911   
           2   341867  325576  ...  568723  406058  199260  401404  423088   
           3   342177  326025  ...  570250  406487  199695  399141  424218   
           4   342100  327364  ...  571860  406419  200200  398613  424049   
...               ...     ...  ...     ...     ...     ...     ...     ...   
2022-12-31 19  289098  271830  ...  523803  381224  197288  444111  363690   
           20  292223  274870  ...  525617  382897  198023  435167  368468   
           21  296167  278586  ...  527764  384559  198068  429620  372154   
           22  300823  280179  ...  531454  385249  198596  423458  376128   
           23  305403  278710  ...  532666  382621  196899  410464  377082   

자치구코드           11620   11650   11680   11710   11740  
기준일        시간                                          
2017-01-01 0   528891  502780  672048  732820  507290  
           1   537046  501029  658925  724738  509894  
           2   538588  495807  653432  722110  510161  
           3   539988  494396  648601  721507  510479  
           4   540018  494538  645242  720435  510177  
...               ...     ...     ...     ...     ...  
2022-12-31 19  440246  485735  700698  762829  495518  
           20  446724  480641  689867  753852  500961  
           21  456023  478041  672511  733852  506973  
           22  462918  473958  645645  720848  513082  
           23  468533  464794  626217  726058  515174  

[52272 rows x 25 columns]

In [10]:
local_people_temp = pd.DataFrame(local_people_temp.stack().unstack('시간'))
local_people_temp

시간                    0       1       2       3       4       5       6   \
기준일        자치구코드                                                           
2017-01-01 11110  283391  240461  231657  227495  226370  225075  226229   
           11140  189272  165315  160031  158427  157119  157601  163923   
           11170  288680  279707  274205  270181  266860  266236  266438   
           11200  313852  311892  311383  312212  312450  313515  313141   
           11215  382976  384657  382445  381230  381829  381414  387030   
...                  ...     ...     ...     ...     ...     ...     ...   
2022-12-31 11620  484463  487124  487545  487414  485863  484374  480484   
           11650  459421  455892  454201  451885  453453  459149  467409   
           11680  656585  648336  642276  637171  632630  633771  641821   
           11710  723550  721368  719291  720087  719438  720262  721306   
           11740  523283  524804  525571  524856  524186  524117  520566   

시간                    7       8       9   ...      14      15      16      17  \
기준일        자치구코드                          ...                                   
2017-01-01 11110  230775  235878  245720  ...  313274  311093  302643  286526   
           11140  170687  178489  189922  ...  240995  237953  230313  221788   
           11170  268129  274735  283308  ...  311744  311993  308684  303508   
           11200  312318  313998  314758  ...  310715  314563  317405  318527   
           11215  382380  381733  379958  ...  379241  383357  388376  391812   
...                  ...     ...     ...  ...     ...     ...     ...     ...   
2022-12-31 11620  473874  467394  458791  ...  430320  428399  428954  430704   
           11650  474338  486020  507418  ...  532910  525779  519522  503607   
           11680  650481  678027  713033  ...  762293  754680  744753  730509   
           11710  720085  720503  723420  ...  751894  759972  767076  769545   
           11740  515920  511133  503848  ...  475679  475830  477245  483607   

시간                    18      19      20      21      22      23  
기준일        자치구코드                                                  
2017-01-01 11110  271633  258441  244637  233384  222604  212971  
           11140  209846  198251  186824  177036  164903  154371  
           11170  298090  294662  290979  284779  275741  259224  
           11200  319038  321274  325111  327972  328504  324973  
           11215  394953  397908  399771  401482  402102  388253  
...                  ...     ...     ...     ...     ...     ...  
2022-12-31 11620  435978  440246  446724  456023  462918  468533  
           11650  491110  485735  480641  478041  473958  464794  
           11680  713589  700698  689867  672511  645645  626217  
           11710  766879  762829  753852  733852  720848  726058  
           11740  488802  495518  500961  506973  513082  515174  

[54450 rows x 24 columns]

In [11]:
local_people_temp = local_people_temp.div(local_people_temp.sum(axis = 1), axis = 0)
local_people_temp

시간                      0         1         2         3         4         5   \
기준일        자치구코드                                                               
2017-01-01 11110  0.045996  0.039028  0.037599  0.036924  0.036741  0.036531   
           11140  0.041112  0.035908  0.034760  0.034412  0.034128  0.034233   
           11170  0.041923  0.040620  0.039821  0.039236  0.038754  0.038663   
           11200  0.041361  0.041103  0.041035  0.041145  0.041176  0.041316   
           11215  0.041346  0.041527  0.041288  0.041157  0.041222  0.041177   
...                    ...       ...       ...       ...       ...       ...   
2022-12-31 11620  0.044071  0.044313  0.044351  0.044339  0.044198  0.044062   
           11650  0.039101  0.038801  0.038657  0.038460  0.038593  0.039078   
           11680  0.039592  0.039094  0.038729  0.038421  0.038147  0.038216   
           11710  0.040962  0.040839  0.040721  0.040766  0.040729  0.040776   
           11740  0.043331  0.043457  0.043521  0.043461  0.043406  0.043400   

시간                      6         7         8         9   ...        14  \
기준일        자치구코드                                          ...             
2017-01-01 11110  0.036718  0.037456  0.038284  0.039882  ...  0.050846   
           11140  0.035606  0.037075  0.038770  0.041253  ...  0.052347   
           11170  0.038693  0.038938  0.039898  0.041143  ...  0.045272   
           11200  0.041267  0.041159  0.041380  0.041480  ...  0.040947   
           11215  0.041783  0.041281  0.041211  0.041020  ...  0.040942   
...                    ...       ...       ...       ...  ...       ...   
2022-12-31 11620  0.043709  0.043107  0.042518  0.041735  ...  0.039145   
           11650  0.039781  0.040371  0.041365  0.043186  ...  0.045356   
           11680  0.038702  0.039224  0.040885  0.042996  ...  0.045966   
           11710  0.040835  0.040766  0.040790  0.040955  ...  0.042567   
           11740  0.043106  0.042721  0.042325  0.041722  ...  0.039389   

시간                      15        16        17        18        19        20  \
기준일        자치구코드                                                               
2017-01-01 11110  0.050492  0.049121  0.046505  0.044088  0.041946  0.039706   
           11140  0.051686  0.050026  0.048175  0.045581  0.043062  0.040580   
           11170  0.045308  0.044828  0.044076  0.043289  0.042792  0.042257   
           11200  0.041455  0.041829  0.041977  0.042044  0.042339  0.042845   
           11215  0.041387  0.041929  0.042300  0.042639  0.042958  0.043159   
...                    ...       ...       ...       ...       ...       ...   
2022-12-31 11620  0.038971  0.039021  0.039180  0.039660  0.040048  0.040637   
           11650  0.044749  0.044216  0.042862  0.041798  0.041341  0.040907   
           11680  0.045507  0.044908  0.044049  0.043029  0.042252  0.041599   
           11710  0.043024  0.043426  0.043566  0.043415  0.043186  0.042678   
           11740  0.039402  0.039519  0.040046  0.040476  0.041032  0.041483   

시간                      21        22        23  
기준일        자치구코드                                
2017-01-01 11110  0.037880  0.036130  0.034566  
           11140  0.038454  0.035819  0.033531  
           11170  0.041356  0.040044  0.037645  
           11200  0.043222  0.043292  0.042826  
           11215  0.043344  0.043410  0.041915  
...                    ...       ...       ...  
2022-12-31 11620  0.041483  0.042111  0.042621  
           11650  0.040686  0.040338  0.039558  
           11680  0.040552  0.038932  0.037761  
           11710  0.041545  0.040809  0.041104  
           11740  0.041981  0.042486  0.042660  

[54450 rows x 24 columns]

In [12]:
local_people_temp = pd.DataFrame(local_people_temp.stack())
local_people_temp = local_people_temp.reset_index()
local_people_temp.columns = ['기준일', '자치구코드', '시간', '총생활인구비율']
local_people_temp

,기준일,자치구코드,시간,총생활인구비율
0,2017-01-01,11110,0,0.045996
1,2017-01-01,11110,1,0.039028
2,2017-01-01,11110,2,0.037599
3,2017-01-01,11110,3,0.036924
4,2017-01-01,11110,4,0.036741
...,...,...,...,...
1306795,2022-12-31,11740,19,0.041032
1306796,2022-12-31,11740,20,0.041483
1306797,2022-12-31,11740,21,0.041981
1306798,2022-12-31,11740,22,0.042486


In [13]:
local_people = pd.merge(local_people, local_people_temp)
local_people

,자치구코드,행정구,기준일,시간,총생활인구수,총생활인구비율
0,11110,종로구,2017-01-01,0,283391,0.045996
1,11140,중구,2017-01-01,0,189272,0.041112
2,11170,용산구,2017-01-01,0,288680,0.041923
3,11200,성동구,2017-01-01,0,313852,0.041361
4,11215,광진구,2017-01-01,0,382976,0.041346
...,...,...,...,...,...,...
1306795,11620,관악구,2022-12-31,23,468533,0.042621
1306796,11650,서초구,2022-12-31,23,464794,0.039558
1306797,11680,강남구,2022-12-31,23,626217,0.037761
1306798,11710,송파구,2022-12-31,23,726058,0.041104
